In [1]:
from pyspark.sql import SparkSession

In [2]:
# Iniciar uma sessão do Spark
spark = SparkSession.builder.appName("CSVLoader").getOrCreate()

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/16 21:53:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Caminho para o arquivo CSV
file_path = "./Cancelled_Diverted_2023.csv"

# Ler o arquivo CSV
df = spark.read.csv(file_path, header=True, inferSchema=True)


In [5]:
# Mostrar o esquema do DataFrame
df.printSchema()

root
 |-- FlightDate: date (nullable = true)
 |-- Day_Of_Week: integer (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Tail_Number: string (nullable = true)
 |-- Cancelled: double (nullable = true)
 |-- Diverted: double (nullable = true)
 |-- Dep_Airport: string (nullable = true)
 |-- Dep_CityName: string (nullable = true)
 |-- DepTime_label: string (nullable = true)
 |-- Dep_Delay: double (nullable = true)
 |-- Dep_Delay_Tag: integer (nullable = true)
 |-- Dep_Delay_Type: string (nullable = true)
 |-- Arr_Airport: string (nullable = true)
 |-- Arr_CityName: string (nullable = true)
 |-- Arr_Delay: double (nullable = true)
 |-- Arr_Delay_Type: string (nullable = true)
 |-- Flight_Duration: double (nullable = true)
 |-- Distance_type: string (nullable = true)
 |-- Delay_Carrier: double (nullable = true)
 |-- Delay_Weather: double (nullable = true)
 |-- Delay_NAS: double (nullable = true)
 |-- Delay_Security: double (nullable = true)
 |-- Delay_LastAircraft: double (nullabl

In [4]:
# Mostrar o DataFrame
df.show()

+----------+-----------+------------+-----------+---------+--------+-----------+--------------------+-------------+---------+-------------+------------------+-----------+--------------------+---------+----------------+---------------+-------------+-------------+-------------+---------+--------------+------------------+
|FlightDate|Day_Of_Week|     Airline|Tail_Number|Cancelled|Diverted|Dep_Airport|        Dep_CityName|DepTime_label|Dep_Delay|Dep_Delay_Tag|    Dep_Delay_Type|Arr_Airport|        Arr_CityName|Arr_Delay|  Arr_Delay_Type|Flight_Duration|Distance_type|Delay_Carrier|Delay_Weather|Delay_NAS|Delay_Security|Delay_LastAircraft|
+----------+-----------+------------+-----------+---------+--------+-----------+--------------------+-------------+---------+-------------+------------------+-----------+--------------------+---------+----------------+---------------+-------------+-------------+-------------+---------+--------------+------------------+
|2023-01-25|          3|Endeavor Air|

In [6]:
# cria uma visualização temporária para usar com comandos sql
df.createOrReplaceTempView("diverted")


In [8]:
sqlDF = spark.sql("SELECT * FROM diverted")

In [9]:
sqlDF.show()

+----------+-----------+------------+-----------+---------+--------+-----------+--------------------+-------------+---------+-------------+------------------+-----------+--------------------+---------+----------------+---------------+-------------+-------------+-------------+---------+--------------+------------------+
|FlightDate|Day_Of_Week|     Airline|Tail_Number|Cancelled|Diverted|Dep_Airport|        Dep_CityName|DepTime_label|Dep_Delay|Dep_Delay_Tag|    Dep_Delay_Type|Arr_Airport|        Arr_CityName|Arr_Delay|  Arr_Delay_Type|Flight_Duration|Distance_type|Delay_Carrier|Delay_Weather|Delay_NAS|Delay_Security|Delay_LastAircraft|
+----------+-----------+------------+-----------+---------+--------+-----------+--------------------+-------------+---------+-------------+------------------+-----------+--------------------+---------+----------------+---------------+-------------+-------------+-------------+---------+--------------+------------------+
|2023-01-25|          3|Endeavor Air|

Datas que ocoreram mais alterações

In [19]:
dataDf = spark.sql("SELECT FlightDate AS `data de voo`, COUNT(*) AS `contagem` FROM diverted GROUP BY FlightDate ORDER BY `contagem` DESC")
dataDf.show()

+-----------+--------+
|data de voo|contagem|
+-----------+--------+
| 2023-02-01|    2147|
| 2023-06-26|    2065|
| 2023-06-27|    1924|
| 2023-01-31|    1842|
| 2023-07-09|    1813|
| 2023-08-07|    1711|
| 2023-07-16|    1570|
| 2023-02-22|    1544|
| 2023-06-25|    1458|
| 2023-09-10|    1392|
| 2023-09-09|    1294|
| 2023-01-11|    1268|
| 2023-09-08|    1220|
| 2023-07-14|    1138|
| 2023-01-30|    1117|
| 2023-03-14|    1049|
| 2023-06-28|    1049|
| 2023-08-20|    1031|
| 2023-08-30|    1029|
| 2023-01-02|     973|
+-----------+--------+
only showing top 20 rows



Dias da semana com mais alterações

In [23]:
mesDf = spark.sql("""
    SELECT 
        CASE 
            WHEN Day_Of_Week = 1 THEN 'Segunda-feira'
            WHEN Day_Of_Week = 2 THEN 'Terça-feira'
            WHEN Day_Of_Week = 3 THEN 'Quarta-feira'
            WHEN Day_Of_Week = 4 THEN 'Quinta-feira'
            WHEN Day_Of_Week = 5 THEN 'Sexta-feira'
            WHEN Day_Of_Week = 6 THEN 'Sábado'
            WHEN Day_Of_Week = 7 THEN 'Domindo'
        END AS `dia da semana`,
        COUNT(*) AS `contagem` 
    FROM diverted 
    GROUP BY Day_Of_Week 
    ORDER BY `contagem` DESC
""")
mesDf.show()

+-------------+--------+
|dia da semana|contagem|
+-------------+--------+
| Quarta-feira|   17461|
|      Domindo|   16921|
|Segunda-feira|   15333|
|  Sexta-feira|   14822|
|  Terça-feira|   14467|
| Quinta-feira|   13784|
|       Sábado|   11700|
+-------------+--------+



Comparação de voos cancelados com desviados

In [29]:
cancelDf = spark.sql("SELECT COUNT(*) AS `Cancelados` FROM diverted WHERE Cancelled LIKE 1.0")
cancelDf.show()

+----------+
|Cancelados|
+----------+
|     87936|
+----------+



In [30]:
desDf = spark.sql("SELECT COUNT(*) AS `Cancelados` FROM diverted WHERE Diverted LIKE 1.0")
desDf.show()

+----------+
|Cancelados|
+----------+
|     16552|
+----------+



Distribuição de voos por linha

In [31]:
linVooDf = spark.sql("SELECT Airline AS `linha de voo`, COUNT(*) AS `contagem` FROM diverted GROUP BY Airline ORDER BY `contagem` DESC")
linVooDf.show()

+--------------------+--------+
|        linha de voo|contagem|
+--------------------+--------+
|Southwest Airline...|   17227|
|American Airlines...|   12473|
|United Air Lines ...|   12180|
| Delta Air Lines Inc|   12055|
|Skywest Airlines ...|   10302|
|    Republic Airways|    8785|
|     JetBlue Airways|    6937|
|    Spirit Air Lines|    5026|
|        Endeavor Air|    4615|
|Frontier Airlines...|    4083|
|        PSA Airlines|    3072|
|American Eagle Ai...|    2793|
|Alaska Airlines Inc.|    2701|
|Hawaiian Airlines...|    1125|
|       Allegiant Air|    1114|
+--------------------+--------+



Aeroportos de destinos

In [33]:
aerDesDf = spark.sql("SELECT Dep_Airport AS `Aeroporto`, COUNT(*) AS `contagem` FROM diverted GROUP BY Dep_Airport ORDER BY `contagem` DESC")
aerDesDf.show()

+---------+--------+
|Aeroporto|contagem|
+---------+--------+
|      DFW|    5223|
|      LGA|    4924|
|      DEN|    4785|
|      EWR|    4179|
|      ORD|    3796|
|      ATL|    3527|
|      JFK|    3428|
|      BOS|    2962|
|      LAS|    2671|
|      MCO|    2523|
|      DCA|    2464|
|      CLT|    2230|
|      LAX|    2093|
|      FLL|    1900|
|      SFO|    1780|
|      MSP|    1740|
|      DTW|    1723|
|      PHX|    1547|
|      IAH|    1500|
|      MDW|    1401|
+---------+--------+
only showing top 20 rows



Aeroportos de origem

In [34]:
aerOriDf = spark.sql("SELECT Arr_Airport AS `Aeroporto`, COUNT(*) AS `contagem` FROM diverted GROUP BY Arr_Airport ORDER BY `contagem` DESC")
aerOriDf.show()

+---------+--------+
|Aeroporto|contagem|
+---------+--------+
|      DFW|    5485|
|      DEN|    5409|
|      LGA|    5108|
|      EWR|    4386|
|      ORD|    4068|
|      ATL|    3516|
|      JFK|    3513|
|      MCO|    2999|
|      BOS|    2952|
|      LAS|    2636|
|      DCA|    2577|
|      CLT|    2194|
|      FLL|    2105|
|      LAX|    1957|
|      SFO|    1756|
|      IAH|    1644|
|      DTW|    1640|
|      MSP|    1628|
|      MIA|    1568|
|      MDW|    1548|
+---------+--------+
only showing top 20 rows



Principais aeronaves

In [36]:
aerDf = spark.sql("""
    SELECT Tail_Number AS `Aeronave`, COUNT(*) AS `contagem` 
    FROM diverted
    WHERE Tail_Number NOT LIKE 0
    GROUP BY Tail_Number
    ORDER BY `contagem` DESC"""
    )
aerDf.show()

+--------+--------+
|Aeronave|contagem|
+--------+--------+
|  N794SK|      81|
|  N355JB|      78|
|  N731YX|      75|
|  N649RW|      73|
|  N740YX|      70|
|  N249JB|      69|
|  N748YX|      68|
|  N306JB|      68|
|  N904XJ|      68|
|  N789SK|      68|
|  N787SK|      67|
|  N914XJ|      66|
|  N915XJ|      66|
|  N375JB|      66|
|  N743SK|      65|
|  N786SK|      64|
|  N733YX|      64|
|  N267JB|      64|
|  N742YX|      64|
|  N796SK|      63|
+--------+--------+
only showing top 20 rows

